In [1]:
import time
import numpy as np

import hoomd
import gsd.hoomd

import coxeter

#### initializing snapshot

In [2]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=59920)
sim.create_state_from_gsd(filename='./DATA/randomize.gsd', frame=-1)

#### integrator

In [3]:
alpha_init=1.0
family323 = coxeter.families.Family323Plus()
particle = family323.get_shape(a=alpha_init, c=0.2*alpha_init+0.8)
verts = particle.vertices/particle.volume**(1/3)

mc = hoomd.hpmc.integrate.ConvexPolyhedron()
mc.shape["A"] = dict(vertices=verts)

#### quick compress

In [4]:
final_volume_fraction = 0.57

initial_box = sim.state.box
final_box = hoomd.Box.from_box(initial_box)
V_particle = 1
final_box.volume = sim.state.N_particles * V_particle / final_volume_fraction
compress = hoomd.hpmc.update.QuickCompress(trigger=hoomd.trigger.Periodic(100),
                                           target_box=final_box)

periodic = hoomd.trigger.Periodic(10)
tune = hoomd.hpmc.tune.MoveSize.scale_solver(moves=['a', 'd'],
                                             target=0.2,
                                             trigger=periodic,
                                             max_translation_move=0.2,
                                             max_rotation_move=0.2)

#### write

In [5]:
logger = hoomd.logging.Logger()
logger.add(mc, quantities=['type_shapes'])

gsd_writer = hoomd.write.GSD(filename='./DATA/compress.gsd',
                             trigger=hoomd.trigger.Periodic(100),
                             mode='wb',
                             filter=hoomd.filter.All(),
                             log=logger)

/Users/lina492375qw1188/opt/miniconda3/envs/patchy/lib/python3.9/site-packages/hoomd/write/gsd.py:183: FutureWarning: log keyword arguments passed to <hoomd.write.gsd.GSD object at 0x10fee0970> is deprecated since v3.9.0. Use logger instead.
  warnings.warn(


#### attaching operations

In [6]:
sim.operations.writers.append(gsd_writer)
sim.operations.integrator = mc
sim.operations.updaters.append(compress)
sim.operations.tuners.append(tune)

#### run simulation

In [7]:
while not compress.complete and sim.timestep < 1e6:
    sim.run(1000)

#### check compress

In [8]:
if not compress.complete:
    raise RuntimeError("Compression failed to complete")

In [9]:
mc.overlaps

0